# 活性化エネルギー予測の評価（Google Colab版）

このノートブックは、ノイズ除去前後のPSDデータから活性化エネルギーを予測し、精度を比較します。

## 実行手順

1. Google Driveをマウント
2. 必要なパッケージをインストール
3. プロジェクトのパスを設定
4. 評価を実行（32000サンプル）
5. 結果を確認


## 1. Google Driveのマウント


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# モジュールの再読み込みを有効化
import importlib
import sys

# 既にインポートされたモジュールを再読み込みできるようにする
def reload_module(module_name):
    """指定されたモジュールを再読み込み"""
    if module_name in sys.modules:
        importlib.reload(sys.modules[module_name])
        print(f'✓ {module_name} を再読み込みしました')
    else:
        print(f'⚠ {module_name} はまだインポートされていません')

print('モジュール再読み込み機能を有効化しました')
print('コードを変更した場合は、このセルを再実行してから評価を実行してください')


## 2. プロジェクトのパス設定

**重要**: 以下の`PROJECT_PATH`を、Google Drive内のプロジェクトフォルダのパスに変更してください。

例: `/content/drive/MyDrive/soturon`


In [ ]:
import os
import sys

# プロジェクトのパスを設定（ここを変更してください）
PROJECT_PATH = '/content/drive/MyDrive/soturon'

# プロジェクトルートをパスに追加
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)

# 作業ディレクトリをプロジェクトルートに変更
os.chdir(PROJECT_PATH)

print(f'プロジェクトパス: {PROJECT_PATH}')
print(f'現在の作業ディレクトリ: {os.getcwd()}')


## 3. 必要なパッケージのインストール


In [ ]:
!pip install scipy numpy torch


## 4. ノイズモジュールのパス設定


In [ ]:
# ノイズモジュールのパスを追加
noise_module_path = os.path.join(PROJECT_PATH, 'ノイズの付与(共通)')
if noise_module_path not in sys.path:
    sys.path.insert(0, noise_module_path)

print(f'ノイズモジュールのパス: {noise_module_path}')
print(f'パスが追加されました: {noise_module_path in sys.path}')

# テストインポート（エラーチェック）
try:
    from add_noise import add_noise_to_interval
    print('ノイズモジュールのインポート成功')
except ImportError as e:
    print(f'ノイズモジュールのインポートエラー: {e}')
    print('このエラーは無視しても問題ない場合があります（後で再試行されます）')


In [ ]:
# add_noise.pyの相対インポートを絶対インポートに修正（Google Colab対応）
import os

add_noise_path = os.path.join(PROJECT_PATH, 'ノイズの付与(共通)', 'add_noise.py')

if os.path.exists(add_noise_path):
    # ファイルを読み込む
    with open(add_noise_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 相対インポートを絶対インポートに置き換え
    if 'from .power_supply_noise import' in content:
        content = content.replace('from .power_supply_noise import', 'from power_supply_noise import')
        content = content.replace('from .interference_noise import', 'from interference_noise import')
        content = content.replace('from .clock_leakage_noise import', 'from clock_leakage_noise import')
        
        # コメントも更新
        if '# 相対インポート' in content:
            content = content.replace('# 相対インポート', '# 絶対インポート（Google Colab対応）')
        
        # ファイルを書き戻す
        with open(add_noise_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print('✓ add_noise.pyを修正しました（相対インポート → 絶対インポート）')
    else:
        print('✓ add_noise.pyは既に修正済みです')
else:
    print(f'⚠ add_noise.pyが見つかりません: {add_noise_path}')


## 5. 評価の実行（32000サンプル）

**利用可能なノイズタイプ**:
- `'power_supply'` - 電源ノイズ
- `'interference'` - 干渉ノイズ
- `'clock_leakage'` - クロックリークノイズ

**設定**:
- サンプル数: 32000（全データ）
- ノイズレベル: 0.3
- 構造化ノイズ: あり

**ノイズタイプを変更する場合**: 以下のセルで`noise_type`パラメータを変更してください。


In [ ]:
# ============================================================
# ノイズタイプを選択（ここを変更してください）
# ============================================================
# 利用可能なノイズタイプ:
#   'power_supply'    - 電源ノイズ
#   'interference'    - 干渉ノイズ
#   'clock_leakage'  - クロックリークノイズ
# ============================================================
NOISE_TYPE = 'power_supply'  # ← ここを変更: 'power_supply', 'interference', 'clock_leakage'
NOISE_LEVEL = 0.3  # ノイズレベル（0.0～1.0）

# 既にインポートされたモジュールをクリア（再インポートのため）
import sys
modules_to_clear = [k for k in sys.modules.keys() if 'add_noise' in k or 'task' in k or 'evaluate_activation_energy' in k or 'dataset' in k]
for module_name in modules_to_clear:
    if module_name in sys.modules:
        del sys.modules[module_name]

# 活性化エネルギー予測フォルダをパスに追加
activation_energy_dir = os.path.join(PROJECT_PATH, '活性化エネルギー予測')
if activation_energy_dir not in sys.path:
    sys.path.insert(0, activation_energy_dir)

# 自己教師あり学習フォルダをパスに追加
ssl_folder = os.path.join(PROJECT_PATH, '自己教師あり学習')
if ssl_folder not in sys.path:
    sys.path.insert(0, ssl_folder)
if os.path.join(ssl_folder, 'task') not in sys.path:
    sys.path.insert(0, os.path.join(ssl_folder, 'task'))

# ノイズモジュールのパスを再度追加（念のため）
noise_module_path = os.path.join(PROJECT_PATH, 'ノイズの付与(共通)')
if noise_module_path not in sys.path:
    sys.path.insert(0, noise_module_path)

# 評価を実行
from evaluate_activation_energy import evaluate_activation_energy_prediction

print('=' * 60)
print('活性化エネルギー予測の評価を開始します')
print('=' * 60)
print(f'サンプル数: 32000（全データ）')
print(f'ノイズタイプ: {NOISE_TYPE}')
print(f'ノイズレベル: {NOISE_LEVEL}')
print('=' * 60)

results = evaluate_activation_energy_prediction(
    model_path='自己教師あり学習/task4_output/best_val_model.pth',
    pickle_path='data_lowF_noise.pickle',
    device='cuda',  # GPUを使用
    num_samples=32000,  # 全データ
    noise_type=NOISE_TYPE,  # ← 選択したノイズタイプを使用
    use_random_noise=False,  # 固定のノイズタイプを使用
    noise_level=NOISE_LEVEL,
)

print('\n' + '=' * 60)
print('実行完了')
print('=' * 60)


## 6. 結果の表示


In [ ]:
if results:
    print(f'\n評価サンプル数: {results["num_samples"]}')
    print(f'\n活性化エネルギー（平均値）:')
    print(f'  正解:')
    print(f'    E_alpha: {results.get("mean_true_E_alpha", "N/A"):.6f} meV')
    print(f'    E_beta: {results.get("mean_true_E_beta", "N/A"):.6f} meV')
    print(f'  ノイズ除去前:')
    print(f'    E_alpha: {results.get("mean_E_alpha_noisy", "N/A"):.6f} meV')
    print(f'    E_beta: {results.get("mean_E_beta_noisy", "N/A"):.6f} meV')
    print(f'  ノイズ除去後:')
    print(f'    E_alpha: {results.get("mean_E_alpha_denoised", "N/A"):.6f} meV')
    print(f'    E_beta: {results.get("mean_E_beta_denoised", "N/A"):.6f} meV')
    print(f'\n誤差（平均値）:')
    print(f'  ノイズありの平均誤差: {results["error_before"]:.6f} meV')
    print(f'  ノイズ除去後の平均誤差: {results["error_after"]:.6f} meV')
    print(f'  改善度: {results["improvement"]:.6f} meV')
    print(f'  改善率: {results["improvement_rate"]:.2f}%')
else:
    print('エラー: 結果が取得できませんでした')


## 7. 結果の保存（オプション）


In [ ]:
import json
from datetime import datetime

if results:
    # 結果をJSON形式で保存
    results_dict = {
        'timestamp': datetime.now().isoformat(),
        'num_samples': results['num_samples'],
        'error_before': float(results['error_before']),
        'error_after': float(results['error_after']),
        'improvement': float(results['improvement']),
        'improvement_rate': float(results['improvement_rate']),
        'mean_true_E_alpha': float(results.get('mean_true_E_alpha', 0)),
        'mean_true_E_beta': float(results.get('mean_true_E_beta', 0)),
        'mean_E_alpha_noisy': float(results.get('mean_E_alpha_noisy', 0)),
        'mean_E_beta_noisy': float(results.get('mean_E_beta_noisy', 0)),
        'mean_E_alpha_denoised': float(results.get('mean_E_alpha_denoised', 0)),
        'mean_E_beta_denoised': float(results.get('mean_E_beta_denoised', 0)),
        'noise_type': 'power_supply',
        'noise_level': 0.3,
    }
    
    output_path = os.path.join(PROJECT_PATH, '活性化エネルギー予測', 'evaluation_results.json')
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results_dict, f, indent=2, ensure_ascii=False)
    
    print(f'\n結果を保存しました: {output_path}')
else:
    print('結果が取得できなかったため、保存をスキップしました')
